### set session

In [1]:
name = 'ANA1'
session = '20231211_d21'
num_trials = 143
threshold_shift = 0.0

### imports

In [2]:
from scipy import io
import numpy as np
import matplotlib.pyplot as plt
import statistics
import pandas as pd
import pickle
import glob # find files in a directory
import re # regex for string parsing

from common.set_matplotlib_settings import set_matplotlib_settings
from common.load_tonedisc_matfile import load_tonedisc_matfile
from speed.calculate_time_differences import calculate_time_differences

In [3]:
PreprocessLeverData_folder = "../Data_Copy/AnalysisData/"+name+"/PreprocessLeverData_"+name+"_"+session+"/"
HitMovements_folder = "../Data_Copy/AnalysisData/"+name+"/HitMovements_"+name+"_"+session+"/"
output_folder = "../Data_Copy/AnalysisData/"+name+"/Speed_"+name+"_"+session+"/"

set_matplotlib_settings()

for mac:

In [4]:
!mkdir {output_folder}

mkdir: ../Data_Copy/AnalysisData/ANA1/Speed_ANA1_20231211_d21/: File exists


for windows:

In [5]:
!mkdir "{output_folder}"

mkdir: ../Data_Copy/AnalysisData/ANA1/Speed_ANA1_20231211_d21/: File exists


### get all indices

In [6]:
leverdata_tone_indices = np.fromfile(PreprocessLeverData_folder+"tone_indices.bin", dtype=np.double)
leverdata_leverpress_indices = np.fromfile(PreprocessLeverData_folder+"leverpress_indices.bin", dtype=np.double)
first_threshold_indices = np.fromfile(HitMovements_folder+"first_threshold_indices.bin", dtype=np.double)
second_threshold_indices = np.fromfile(HitMovements_folder+"second_threshold_indices.bin", dtype=np.double)
third_threshold_indices = np.fromfile(HitMovements_folder+"third_threshold_indices.bin", dtype=np.double)

trial_frequencies = np.fromfile(PreprocessLeverData_folder+"trial_frequencies.bin", dtype=np.double)

movement_informations = np.load(HitMovements_folder+"leverpress_informations.npy")

### calculate reaction times as time from tone index to first threshold

In [7]:
reaction_times = calculate_time_differences(leverdata_tone_indices, first_threshold_indices, movement_informations, trial_frequencies)

print("mean reaction time:", np.mean(reaction_times), "s")
np.save(output_folder+"reaction_times", reaction_times)

mean reaction time: 1.9750576223223895 s


### Calculate time from first threshold to second threshold (rise time)

In [8]:
rise_times = calculate_time_differences(first_threshold_indices, second_threshold_indices, movement_informations, trial_frequencies)

print("mean rise time:", np.mean(rise_times), "s")
np.save(output_folder+"rise_times", rise_times)

mean rise time: 0.03476589739473704 s


### Calculated time from second threshold to third threshold (return times)

In [9]:
return_times = calculate_time_differences(second_threshold_indices, third_threshold_indices, movement_informations, trial_frequencies)

print("mean return times:", np.mean(return_times), "s")
np.save(output_folder+"return_times", return_times)

mean return times: 0.2297162126343111 s
